# Ch4. Working with Databases

<div id="toc"></div>

## Unit17_Setting Up a MySQL Database

In [ ]:
c:\myProject> mysql -u root -p
Enter password:
Welcome to the MySQL monitor. Commands end with ; or \g.
«More mysql output»
mysql>

In [ ]:
CREATE USER 'dsuser' @'localhost' IDENTIFIED BY 'badpassw0rd' ;

In [ ]:
CREATE DATABASE dsdb;

In [ ]:
GRANT ALL ON dsdb.* TO 'dsuser' @'localhost' ;

In [ ]:
c:\myProject> mysql -u dsuser -p dsdb
Enter password:
Welcome to the MySQL monitor. Commands end with ; or \g.
«More mysql output»
mysql>

In [ ]:
USE dsdb;
CREATE TABLE employee (empname TINYTEXT, salary FLOAT, hired DATE);

In [ ]:
DROP TABLE employee;

In [ ]:
CREATE TABLE employee (id INT PRIMARY KEY AUTO_INCREMENT,
                       updated TIMESTAMP, empname TINYTEXT NOT NULL, salary FLOAT NOT NULL,
                       hired DATE);

In [ ]:
ALTER TABLE employee ADD INDEX(hired);

In [ ]:
DROP INDEX hired ON employee;

In [ ]:
ALTER TABLE employee ADD UNIQUE(empname(255));

## Unit18_Using a MySQL Database: Command Line

#### Insertion

In [ ]:
INSERT INTO employee VALUES( NULL, NULL,"John Smith",35000,NOW());

In [ ]:
INSERT INTO employee VALUES( NULL, NULL,"John Smith",35000,NOW());

In [ ]:
INSERT IGNORE INTO employee VALUES( NULL, NULL,"John Smith",35000,NOW())

#### Deletion

In [ ]:
-- Remove John Smith if he is low-paid
DELETE FROM employee WHERE salary<11000 AND empname="John Smith";
-- Remove everyone
DELETE FROM employee;

In [ ]:
DELETE FROM employee WHERE id=387513;

#### Mutation

In [ ]:
-- Reset all recent hires' salary
UPDATE employee SET salary=35000 WHERE hired=CURDATE();
-- Increase John Smith's salary again
UPDATE employee SET salary=salary+1000 WHERE empname="John Smith";

#### Selection

In [ ]:
SELECT empname,salary FROM employee WHERE empname="John Smith";

In [ ]:
SELECT empname,salary FROM employee;

In [ ]:
SELECT * FROM employee WHERE hired>='2000-01-01' ORDER BY salary DESC;

In [ ]:
SELECT (hired>'2001-01-01' ) AS Recent, AVG(salary)
FROM employee
GROUP BY (hired>'2001-01-01' );

In [ ]:
SELECT AVG(salary),MIN(hired),MAX(hired) FROM employee
GROUP BY YEAR(hired)
HAVING MIN(hired)>'2001-01-01' ;

#### Join

In [ ]:
-- Prepare and populate another table
CREATE TABLE position (eid INT, description TEXT);
INSERT INTO position (eid,description) VALUES (6, 'Imposter' ),
(1, 'Accountant' ),(4, 'Programmer' ),(5, 'President' );
ALTER TABLE position ADD INDEX(eid);

-- Fetch the joined data
SELECT employee.empname,position.description
FROM employee,position WHERE employee.id=position.eid
ORDER BY position.description;

## Unit19_Using a MySQL Database: PyMySQL

In [ ]:
conn = pymysql.connect(host="localhost" , port=3306,
                       user="dsuser" , passwd="badpassw0rd" , db="dsdb" )
cur = conn.cursor()

In [ ]:
query = '''
SELECT employee.empname,position.description
FROM employee,position WHERE employee.id=position.eid
ORDER BY position.description
'''
n_rows = cur.execute(query)

In [ ]:
results = list(cur.fetchall())

http://quickbase.intuit.com/articles/timeline-of-database-history

## Unit20_Taming Document Stores: MongoDB

In [ ]:
import pymongo as mongo
# Default initialization
client1 = mongo.MongoClient()
# Explicit host and port
client2 = mongo.MongoClient( "localhost" , 27017)
# Explicit host and port as a URI
client3 = mongo.MongoClient( "mongodb://localhost:27017/" )
Once the client establishes a connection to the database server, select the
active database and then the active collection. You can use either the object
oriented ( “dotted ”) or dictionary-style notation. If the selected database or
collection do not exist, the server will create them at once:
# Two ways to create/select the active database
db = client1.dsdb
db = client1[ "dsdb" ]
# Two ways to create/select the active collection
people = db.people
people = db[ "people" ]

http://couchdb.apache.org

In [ ]:
person1 = {"empname" : "John Smith" , "dob" : "1957-12-24" }
person2 = {"_id" : "XVT162" , "empname" : "Jane Doe" , "dob" : "1964-05-16" }
person_id1 = people.insert_one(person1).inserted_id

In [ ]:
# Note the new "_id" field!
person1

In [ ]:
person_id2 = people.insert_one(person2).inserted_id

In [ ]:
persons = [{"empname" : "Abe Lincoln" , "dob" : "1809-02-12" },
           {"empname" : "Anon I. Muss" }]
result = people.insert_many(persons)
result.inserted_ids

In [ ]:
everyone = people.find()
list(everyone)

In [ ]:
list(people.find({"dob" : "1957-12-24" }))

In [ ]:
people.find_one()

In [ ]:
people.find_one({"empname" : "Abe Lincoln" })

In [ ]:
people.find_one({"_id" : "XVT162" })

In [ ]:
people.count()

In [ ]:
people.find({"dob" : "1957-12-24" }).count()

In [ ]:
people.find().sort( "dob" )

In [ ]:
result = people.delete_many({"dob" : "1957-12-24" })
result.deleted_count

## Your Turn